# Time Range Annotation

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np; np.random.seed(0)
import pandas as pd
from scipy.stats import zscore
import string

import colorcet as cc
import holoviews as hv; hv.extension('bokeh')
from holoviews.plotting.links import RangeToolLink
from holoviews.operation.datashader import rasterize
from holoviews import Dataset
from bokeh.models import HoverTool
from holonote.annotate import Annotator
from holonote.app import PanelWidgets
import panel as pn
pn.extension('tabulator')

## Create Multi-Channel Timeseries Plot

We'll use the the [small-data approach](./small_multi-chan-ts.ipynb), but the annotations should work with any of the demonstrated approaches.

### Generate fake data

In [ ]:
n_channels = 8
n_seconds = 300
fs = 256  # Sampling frequency

init_freq = .01  # Initial sine wave frequency in Hz
freq_inc = 2/n_channels  # Frequency increment
amplitude = 1

total_samples = n_seconds * fs
time = np.linspace(0, n_seconds, total_samples)
channels = [f'CH {i}' for i in range(n_channels)]
groups = ['EEG'] * (n_channels // 2) + ['MEG'] * (n_channels - n_channels // 2)

data = np.array([amplitude * np.sin(2 * np.pi * (init_freq + i * freq_inc) * time)
                 for i in range(n_channels)])
print(f'shape: {data.shape} (n_channels, samples) ')

### Create Plot

In [ ]:
time_dim = hv.Dimension('Time', unit='s')
amplitude_dim = hv.Dimension('Amplitude', unit='µV')

curves = []
for group, channel, channel_data in zip(groups, channels, data):
    ds = Dataset((time, channel_data), [time_dim, amplitude_dim])
    curve = hv.Curve(ds, time_dim, amplitude_dim, group=group, label=f'{channel}')
    curve.opts(
        subcoordinate_y=True,
        subcoordinate_scale=.75,
        color="black",
        line_width=1,
        tools=['hover'],
        hover_tooltips=[("Group", "$group"), ("Channel", "$label"), "Time", "Amplitude"],
        apply_hard_bounds=True,
        )
    curves.append(curve)

curves_overlay = hv.Overlay(curves, kdims=["Channel", time_dim])

curves_overlay = curves_overlay.opts(
    xlabel="Time (s)",
    ylabel="Channel",
    show_legend=False,
    padding=0,
    responsive=True,
    shared_axes=False,
    min_height=400,
)

# Minimap
y_positions = range(len(channels))
yticks = [(i, ich) for i, ich in enumerate(channels)]
z_data = zscore(data, axis=1)
minimap = rasterize(hv.Image((time, y_positions, z_data), [time_dim, "Channel"], amplitude_dim))
minimap = minimap.opts(
    cmap="RdBu_r",
    colorbar=False,
    xlabel='',
    alpha=0.5,
    yticks=[yticks[0], yticks[-1]],
    height=150,
    responsive=True,
    default_tools=[],
    shared_axes=False,
)

# Link the curves plot to minimap
link = RangeToolLink(minimap, curves_overlay, axes=["x", "y"],
              boundsy=(-.5, 3.5),
              boundsx=(0, time[len(time)//3])
             )

plot = (curves_overlay + minimap).cols(1)


## Time-Range Annotation

### Create fake time range annotations

In [ ]:
def create_range_annotations(n_total_seconds: int, n_categories: int, 
                             n_total_annotations: int, duration: int = 1) -> pd.DataFrame:

    
    start_times = np.sort(np.random.randint(0, n_total_seconds - duration, n_total_annotations))
    
    # Ensure the annotations are non-overlapping
    for i in range(1, len(start_times)):
        if start_times[i] < start_times[i-1] + duration:
            start_times[i] = start_times[i-1] + duration
    end_times = start_times + duration
    categories = np.random.choice(list(string.ascii_uppercase)[:n_categories], n_total_annotations)
    
    df = pd.DataFrame({
        'start': start_times,
        'end': end_times,
        'category': categories
    })
    df['category'] = df['category'].astype('category')
    df = df.sort_values('start')
    return df

np.random.seed(3)
n_categories = 4
n_total_annotations = 50
annotations_df = create_range_annotations(n_seconds, n_categories, n_total_annotations)
unique_categories = list(annotations_df['category'].unique()) + ['other']
unique_categories.sort()
print(unique_categories)
annotations_df.head()

## Create and Populate Annotator Instance

First, define the Annotator instance. We'll specify the primary annotation 'Time' range type as float and then list the field we want to include, 'category'

In [ ]:
annotator = Annotator({"Time": float}, fields=["category"])
print('spec:', annotator.spec)
print('fields:', annotator.fields)

Next, we'll add our fake annotations to the annotator table

In [ ]:
annotator.define_annotations(annotations_df, Time=("start", "end"))
annotator.df.head()

## Define styling

Groupby the `category` to apply default colors per category

In [ ]:
annotator.groupby = "category"

Alternatively, we can explicitly map colors to categories

In [ ]:
color_map = dict(zip(unique_categories, cc.glasbey[:len(unique_categories)]))
annotator.style.color = hv.dim("category").categorize(categories=color_map, default="grey")
annotator.style.color

## Create Widgets

In [ ]:
annotator.style.edit_color = "grey"

In [ ]:
widget_select = pn.widgets.MultiSelect(value=unique_categories, options=unique_categories,)
annotator.visible = widget_select
annotator_tools = pn.panel(PanelWidgets(annotator, {"category": unique_categories}))

In [ ]:
df = annotator.df.rename(columns={'start[Time]': 'start', 'end[Time]': 'end'}).reset_index(drop=True)
hn_tabulator = pn.widgets.Tabulator(df,
                     pagination='local',
                     page_size=10,
                     layout='fit_columns',
                     editors={'index': None,
                              'category': {'type': 'list', 'valuesLookup': True},
                              'start': {'type': 'float'},
                              'end': {'type': 'float'},
                             },
                    sizing_mode='stretch_width',
                    header_align='center',
                    text_align='center',  
                    )

In [ ]:
show_hide = ('Show/Hide',
     pn.Column(
         widget_select,
     ))

table = ('Table',
     pn.Column(
         pn.Row(
             pn.widgets.Select(name='Select all of', value=unique_categories[0], options=unique_categories, width=100),
             pn.widgets.Button(name='Deselect', icon='deselect', button_type='primary', align='end'),
         ),
         hn_tabulator,
         pn.Row(
             pn.widgets.Button(name='Delete', icon='skull', button_type='danger', align='end'),
             pn.widgets.Button(name='Edit', icon='category', button_type='warning', align='end', 
                               description='pop up modal to edit category of selected rows')
         )
     ))

create_new = ('Create New',
     pn.Column(
         pn.widgets.Select(name='category', value=unique_categories[0], options=unique_categories),
         pn.Row(
             pn.widgets.FloatInput(name='start', width=80),
             pn.widgets.FloatInput(name='end', width=80),
             pn.widgets.Button(name='Add', width=80, align='end', icon='hand-click', button_type='primary')
         ),
     ))

save_revert = ('Save/Revert',
     pn.Column(
         pn.pane.Alert('⚠ There are unsaved changes', alert_type='warning', stylesheets=[':host(.alert) { padding: 0.1rem 1.25rem;}']),
         pn.Row(
             pn.widgets.Button(name='Save Changes', icon='device-floppy', button_type='success', align='end'),
             pn.widgets.Button(name='Revert Changes', icon='arrow-back-up', button_type='warning', align='end')
         )
     ))

old_gui = ('Old GUI',
     pn.Column(
         annotator_tools,
     ))

In [ ]:
widgets = pn.WidgetBox(
    pn.pane.Markdown('## HoloNote', align='center'),
    pn.Tabs(('Time', pn.Accordion(show_hide, table, create_new, save_revert, old_gui, sizing_mode='stretch_width', active=[0,1,2,3]))),
)

## Display Widgets with Plot

Demetris
- Reselecting the pan tool activates it.. box select is shadow-activated at first. Open holoviews/holonote issue.
- why does including the minimap limit the height of the subcoord plot? File holoviews issue
- Why can't we edit the start/end cells as floats? File panel issue
- Why can't we see the tabulator page buttons in the served app? could try reducing the font size, otherwise file issue
- File simon's issues as holonote issues.
- add the Annotate1D one to the batch issue for this sprint.
  
Simon:
- Add coloring to widget_select (annotator.visible widget)
- pin color to category, rather than being based on which are initially displayed
- kdims: avoid use of `get_element`. we should be able to just `app * annotator`, matching kdims.
- overlay opts: Why do we have to re-apply some the opts to the new composite curves overlay but not the minimap? replacing annotations_overlay with `dmap = (hv.DynamicMap(lambda: hv.Curve([])))` replicates the issue. Might be related to holoviews #5441
- Create functional Annotate1D batch app in Holonote based on this widget mockup. including:
  - synced updates with the tabulator table (adding, editing, deleting annotations)
  - indicator of uncommitted changes (e.g. color or mark the uncommited rows of the table)

In [ ]:
app_w_annotations = pn.Column((annotator * plot).cols(1), min_height=500)
app_w_annotations


In [ ]:
# annotations_overlay = annotator.get_element("Time").opts(show_legend=False, responsive=True)

# curves_annotations = (curves_overlay * annotations_overlay).opts(
#     ylabel="Channel",
#     show_legend=False,
#     responsive=True,
# )

# with https://github.com/holoviz/holonote/pull/98 we can do this instead of the above:
app_w_annotations = pn.Column((annotator * plot).cols(1), min_height=500)

# app_w_annotations = pn.Column((curves_annotations + minimap * annotations_overlay).cols(1), min_height=800)
widget_app_annotations = pn.Row(pn.Column(widgets, width=300, margin=(40,10)), app_w_annotations)
widget_app_annotations

## Served app

TODO: how to fit the tabulator table in the sidebar of the served app

In [ ]:
template = pn.template.BootstrapTemplate(
    title='Multi-Channel Timeseries App - Time Annotations ',
    main = [app_w_annotations],
    sidebar=[widgets],
)
template.servable();